In [1]:
import tensorflow as tf
import pandas as pd
import keras
train = pd.read_csv("dataset_train_set.csv")
test = pd.read_csv("dataset_dev_set.csv")

Using TensorFlow backend.


In [2]:
X_train = train.drop('target',axis=1)
y_train = train['target']
X_test = test.drop('target',axis=1)
y_test = test['target']

In [3]:
X_train = X_train[['aPosLS','hPosLS','hGD5','aSpent','aPtsPG','hSPG','hGCPG']].copy()
X_test = X_test[['aPosLS','hPosLS','hGD5','aSpent','aPtsPG','hSPG','hGCPG']].copy()

In [4]:
X_train.head()

,aPosLS,hPosLS,hGD5,aSpent,aPtsPG,hSPG,hGCPG
0,18,18,0.0,18.07,0.0,0.0,0.0
1,9,5,0.0,8.92,0.0,0.0,0.0
2,12,14,0.0,29.52,0.0,0.0,0.0
3,15,16,0.0,0.00,0.0,0.0,0.0
4,13,3,0.0,31.03,0.0,0.0,0.0


In [5]:
print(X_train.shape)
X_train_norm = tf.keras.utils.normalize(X_train.values,axis=1)
X_test_norm = tf.keras.utils.normalize(X_test.values,axis=1)
X_train_norm[1000]

(4180, 7)


array([ 0.32600457,  0.39120548, -0.01304018,  0.2640637 ,  0.08410918,
        0.81227298,  0.06259288])

In [6]:
y_train.value_counts()

2    1971
0    1120
1    1089
Name: target, dtype: int64

In [7]:
whome = 1
wdraw = 1971/1089
waway = 1971/1120
print("{},{},{}".format(whome,wdraw,waway))

1,1.8099173553719008,1.7598214285714286


In [8]:
class_weights = {0:1.7598,
                 1:1.8099,
                 2:1}

In [9]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(3, activation = tf.nn.softmax))

In [10]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [11]:
model.fit(X_train_norm, y_train.values, epochs=50, validation_data=(X_test_norm,y_test), class_weight=class_weights)

Train on 4180 samples, validate on 1520 samples
Epoch 1/50
4180/4180 [==============================] - 1s 159us/step - loss: 1.0970 - acc: 0.3404 - val_loss: 1.0865 - val_acc: 0.4507
Epoch 2/50
4180/4180 [==============================] - 0s 72us/step - loss: 1.0787 - acc: 0.4502 - val_loss: 1.0607 - val_acc: 0.5072
Epoch 3/50
4180/4180 [==============================] - 0s 61us/step - loss: 1.0622 - acc: 0.4636 - val_loss: 1.0396 - val_acc: 0.5184
Epoch 4/50
4180/4180 [==============================] - 0s 65us/step - loss: 1.0522 - acc: 0.4727 - val_loss: 1.0380 - val_acc: 0.5072
Epoch 5/50
4180/4180 [==============================] - 0s 62us/step - loss: 1.0470 - acc: 0.4770 - val_loss: 1.0291 - val_acc: 0.5026
Epoch 6/50
4180/4180 [==============================] - 0s 67us/step - loss: 1.0440 - acc: 0.4720 - val_loss: 1.0299 - val_acc: 0.4908
Epoch 7/50
4180/4180 [==============================] - 0s 63us/step - loss: 1.0413 - acc: 0.4718 - val_loss: 1.0179 - val_acc: 0.5092
Epoch 

In [12]:
val_loss, val_acc = model.evaluate(X_test_norm, y_test)
print(val_loss, val_acc)

1520/1520 [==============================] - 0s 53us/step
1.0046399154161152 0.5151315789473684


In [13]:
predictions = model.predict_classes(X_test_norm)
preds = list(predictions)
pd.Series(list(predictions)).value_counts()

2    692
0    520
1    308
dtype: int64

In [14]:
predictions[0]

1

In [15]:
correct_pred_dict = {0:0,1:0,2:0}
for i in range(len(preds)):
    pred = preds[i]
    real = y_test[i]
    if pred == real:
        correct_pred_dict[pred] += 1
correct_pred_dict

{0: 251, 1: 102, 2: 430}

In [16]:
outcomes_incorrectly_caught_dict = {0:0,1:0,2:0}
for i in range(len(preds)):
    pred = preds[i]
    real = y_test[i]
    if pred != real:
        outcomes_incorrectly_caught_dict[real] += 1
outcomes_incorrectly_caught_dict

{0: 209, 1: 270, 2: 258}